# RANDOM FOREST F90 SERIAL

- Uses **PARF** - Parallel random forest (RF) algorithm, MPI-enabled, implemented in Fortran, CLI. By Goran Topić and Tomislav Šmuc, from Ruđer Bošković Institute, Croatia : https://www.irb.hr/eng/Scientific-Support-Centres/Centre-for-Informatics-and-Computing/Projects2/IT-projects/PARF

- Dataset from : https://www.kaggle.com/sakhawat18/asteroid-dataset (converted to arff using Weka. Attributes dropped: id, spkid, full_name, pdes, name, prefix, orbit_id, equinox. The class attribute was placed in the last position.)

- Instances: train: 66.000, test: 34.000

#### Copy files to /scratch

In [11]:
%%bash
SCR=/scratch${PWD#"/prj"}
mkdir $SCR
cp -r parf/datasets $SCR
cp parf/parf/parf $SCR
cp parf/parf/parf-s $SCR

#### Slurm batch script

In [6]:
%%writefile rfs6.srm
#!/bin/bash
#SBATCH --job-name rfs6         # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:05:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
echo '-- modules ----------------------------'
SCR=/scratch${PWD#"/prj"}
DT1=datasets/asteroid-train-66k.arff
DT2=datasets/asteroid-test-34k.arff
EXEC="parf-s -t "$DT1" -a "$DT2

# Start
echo '-- run --------------------------------'
echo '$ srun -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Writing rfs6.srm


<hr style="height:10px;border-width:0;background-color:green">

## Run

In [7]:
%%bash
sbatch rfs6.srm
sbatch rfs6.srm
sbatch rfs6.srm

Submitted batch job 1344896
Submitted batch job 1344897
Submitted batch job 1344898


In [2]:
%%bash
cat /scratch${PWD#"/prj"}/slurm-1344896.out
cat /scratch${PWD#"/prj"}/slurm-1344897.out
cat /scratch${PWD#"/prj"}/slurm-1344898.out

- Job ID: 1344896
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: rf
- List of nodes allocated to the job: sdumont1278
-- modules ----------------------------
-- run --------------------------------
$ srun -n 1 asteroid-test-34k.arff
-- output -----------------------------
Trainset classification error is   0.06% of   66000 (kappa: 0.9908 )
 Testset classification error is   0.62% of   34000 (kappa: 0.9064 )
T: 136.8559  |  N: 1
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1344897
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: rf
- List of nodes allocated to the job: sdumont1437
-- modules ----------------------------
-- run --------------------------------
$ srun -n 1 asteroid-test-34k.arff
-- output -----------------------------
Trainset classification error is   0.06% of   66000 (kappa: 0.9908 )
 Testset classification error is   0.58% of   34000 (kappa: 0.9118 )
T: 140.7614  | 

In [9]:
! squeue -u $(whoami) -h -t pending,running -r | wc -l

33


In [10]:
! squeue --partition=cpu_small -h -t pending,running -r | wc -l

385


In [8]:
! squeue -n rfs6 -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1344896  cpu_small  PD  0:00     1    1
           1344897  cpu_small  PD  0:00     1    1
           1344898  cpu_small  PD  0:00     1    1


In [1]:
! squeue -n rfs6 -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


<hr style="height:10px;border-width:0;background-color:red">